In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
trad_data = pd.read_csv('trad_data.csv')
# trad_data = trad_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'])
# trad_data = trad_data.drop(columns=['TEAM_CITY', 'NICKNAME', 'COMMENT', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
#        'OREB', 'DREB', 'STL', 'BLK', 'TO', 'PF','PLUS_MINUS', 'START_POSITION'])
# trad_data = trad_data.drop_duplicates()

/var/folders/1j/ycj6sm297qj2d2b1b3pb8k980000gn/T/ipykernel_3734/2466255697.py:1: DtypeWarning: Columns (3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  trad_data = pd.read_csv('trad_data.csv')


In [4]:
trad_data['START_POSITION']

0           F
1           F
2           C
3           G
4           G
         ... 
484382    NaN
484383    NaN
484384    NaN
484385    NaN
484386    NaN
Name: START_POSITION, Length: 484387, dtype: object

In [63]:
trad_data['AVG_PTS'] = trad_data.groupby('PLAYER_ID')['PTS'].transform('median')
trad_data = trad_data.dropna()
top_players = trad_data.sort_values(by='AVG_PTS', ascending=False).groupby(['GAME_ID', 'TEAM_ID']).head(5)
top_players = top_players.groupby(['GAME_ID', 'TEAM_ID']).filter(lambda x: len(x) == 5)

In [64]:
top_players['OVER'] = top_players['PTS'] > top_players['AVG_PTS']
top_players['RANK'] = top_players.groupby(['GAME_ID', 'TEAM_ID'])['AVG_PTS'].rank(method='first', ascending=False)
pivoted = top_players.pivot_table(index=['GAME_ID', 'TEAM_ID'], columns='RANK', values='OVER', aggfunc='first')
pivoted = pivoted.applymap(lambda x: 'over' if x else 'under')
pivoted['OUTCOME'] = pivoted[1].astype(str) + '_' + pivoted[2].astype(str) + '_' + pivoted[3].astype(str) + '_' + pivoted[4].astype(str) + '_' + pivoted[5].astype(str)
outcome_counts = pivoted['OUTCOME'].value_counts()

/var/folders/1j/ycj6sm297qj2d2b1b3pb8k980000gn/T/ipykernel_74099/906401800.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivoted = pivoted.applymap(lambda x: 'over' if x else 'under')


In [65]:
print(outcome_counts)

OUTCOME
over_over_under_under_under      1474
over_over_over_under_under       1431
over_under_over_under_under      1383
over_under_under_under_under     1327
under_over_over_under_under      1286
under_under_under_under_under    1284
over_over_under_under_over       1274
over_under_over_over_under       1253
over_over_under_over_under       1235
under_over_over_over_under       1222
over_over_over_over_under        1214
under_over_under_under_under     1205
under_under_over_over_under      1195
under_over_over_under_over       1180
over_under_under_over_under      1177
over_under_under_under_over      1173
over_under_over_under_over       1171
under_under_over_under_under     1126
under_over_under_over_under      1126
over_over_over_under_over        1112
over_over_under_over_over        1096
under_over_under_over_over       1089
under_over_under_under_over      1088
under_under_over_over_over       1088
under_under_under_over_over      1087
under_under_over_under_over      1073
over

In [66]:
pivoted2 = top_players.pivot_table(index=['GAME_ID', 'TEAM_ID'], columns='RANK', values='PTS')
correlation_matrix = pivoted2.corr()
print(correlation_matrix)

RANK       1.0       2.0       3.0       4.0       5.0
RANK                                                  
1.0   1.000000  0.018298 -0.048443 -0.092324 -0.092245
2.0   0.018298  1.000000 -0.048373 -0.096570 -0.088341
3.0  -0.048443 -0.048373  1.000000 -0.045470 -0.083868
4.0  -0.092324 -0.096570 -0.045470  1.000000 -0.038332
5.0  -0.092245 -0.088341 -0.083868 -0.038332  1.000000
